### This dataset consists of 'circles' (or 'friends lists') from Facebook. Facebook data was collected from survey participants using this Facebook app. The dataset includes node features (profiles), circles, and ego networks.

http://snap.stanford.edu/data/egonets-Facebook.html

In [1]:
#setup
from graph_tool.all import * 
import numpy as np
import math
import time

file_path = "/home/jawa/Datasets/Email-Enron.txt"

In [2]:
# initialization
g = Graph(directed=True)

neighbours = {}
vertex_id = {}
vertex_label = {}

#this time, we watn to add a edge in each direction for running pagerank and other centrality metrics

with open(file_path, encoding="UTF-8") as f:
    for line in f:
        if line.startswith("#"):
            continue
        l = line.replace("\n", "").split("\t")
        a = l[0]
        b = l[1]

        # get the vertexes
        if a not in vertex_label:
            v_a = g.add_vertex()
            vertex_label[a] = v_a
            vertex_id[g.vertex_index[v_a]] = a
        else:
            v_a = g.vertex(vertex_label[a])
        if b not in vertex_label:
            v_b = g.add_vertex()
            vertex_label[b] = v_b
            vertex_id[g.vertex_index[v_b]] = b
        else:
            v_b = g.vertex(vertex_label[b])
        # add the edges
        g.add_edge(v_a, v_b)
    

In [3]:
# degree stats
degrre_data = {}
degrre_data["out"] = []
degrre_data["in"] = []
counter = 0
for v in g.vertices():
        counter += 1
        degrre_data["out"].append(v.out_degree())
        degrre_data["in"].append(v.in_degree())
    
max_out = max(degrre_data["out"])
min_out = min(degrre_data["out"])
average_out = np.average(degrre_data["out"])
sd_out = np.std(degrre_data["out"])
print("Out degree stats -> max value = " + str(max_out) + " / min value = " + str(min_out) + " / average = " + str(average_out) + " / SV = " + str(sd_out))

Out degree stats -> max value = 1383 / min value = 1 / average = 10.0202223918 / SV = 36.100004407


In [4]:
# distance stats
#shortest_distance = graph_tool.topology.shortest_distance(g)
#global_avg = []
#for dist in shortest_distance:
#    global_avg.append(np.average(dist))
pseudo_diameter = graph_tool.topology.pseudo_diameter(g)

# there are non connect components in the graph, so we cant just calculate the avverage shortest distance
#print("Shortest Distance Average = " + str(np.average(global_avg)) + " / Diameter = " + str(pseudo_diameter))
print("Diameter = " + str(pseudo_diameter))

Diameter = (13.0, (<Vertex object with index '29423' at 0x7ff55aa447c8>, <Vertex object with index '33678' at 0x7ff55aa44750>))


In [5]:
# clustering stats
global_clustering = graph_tool.clustering.global_clustering(g)

print ("Clustering Average = " + str(np.average(global_clustering)) + " / " + "Clustering SV = " + str(np.std(global_clustering)))

Clustering Average = 0.0460164089095 / Clustering SV = 0.0389887853106


In [6]:
# connected componet stats
comp, hist = label_components(g)

labels = list(set(comp.a))
largest = label_largest_component(g)
l = GraphView(g, vfilt=largest)

print("Connected components = " + str(len(labels)) + " / Edges in Largest component = " + str(l.num_edges()) + " / Vertices in Largest component = " + str(l.num_vertices()))

Connected components = 1065 / Edges in Largest component = 361622 / Vertices in Largest component = 33696


In [7]:
start = time.time()
# vertex closeness stats
g_closeness = closeness(g)
c_list = []
for vc in g_closeness.a:
    if not math.isnan(vc):
        c_list.append(vc)

end = time.time()
print("Time Taken: " + str(end - start) +"s")
print("Closeness stats -> max value = " + str(max(c_list)) + " / min value = " + str(min(c_list)) + " / average = " + str(np.average(c_list)) + " / SV = " + str(np.std(c_list)))

closeness_dic   = {}
zero_closeness = 0
for i in range(0, len(g_closeness.a)):
    if (g.vertex(i).out_degree() != 0):
        closeness_dic[g.vertex_index[i]] = g_closeness.a[i]
    else:
        zero_closeness += 1

Time Taken: 120.4083731174469s
Closeness stats -> max value = 1.0 / min value = 0.114171961033 / average = 0.307049701751 / SV = 0.190206013781


In [8]:
# betweenness stats
start=time.time()
g_betweenness, edge_betweenness = betweenness(g)
v_b = list(g_betweenness)
end = time.time()
print("Time Taken: " + str(end - start) +"s")
print("Betweenness stats -> max value = " + str(max(v_b)) + " / min value = " + str(min(v_b)) + " / average = " + str(np.average(v_b)) + " / SV = " + str(np.std(v_b)))

betweenness_dic   = {}
zero_betweenness = 0
for i in range(0, len(g_betweenness.a)):
    if (g.vertex(i).out_degree() != 0):
        betweenness_dic[g.vertex_index[i]] = g_betweenness.a[i]
    else:
        zero_betweenness += 1

Time Taken: 152.6016399860382s
Betweenness stats -> max value = 0.0648511760831395 / min value = 0.0 / average = 6.95367656148e-05 / SV = 0.000878765393182


In [9]:
start=time.time()
# degree centrality metrics
out_dic = {}
in_dic  = {}
zero_out = 0
zero_in  = 0
v_num = len(vertex_id.keys())
for v in g.vertices():
    if (v.out_degree() == 0):
        zero_out += 1
    else:
        out_dic[g.vertex_index[v]] = 1.0* v.out_degree() / (v_num - 1)
    if (v.in_degree() == 0):
        zero_in += 1
    else:
        in_dic[g.vertex_index[v]]  = v.in_degree() / (v_num - 1)
end = time.time()
print("Time Taken: " + str(end - start) +"s")

Time Taken: 0.6195271015167236s


In [10]:
start=time.time()
# katz
g_katz = graph_tool.all.katz(g)
katz_dic = {}
for i in range(0, len(g_katz.a)):
    katz_dic[g.vertex_index[i]] = g_katz.a[i]
end = time.time()
print("Time Taken: " + str(end - start) +"s")

/usr/lib/python3/dist-packages/graph_tool/centrality/__init__.py:765: RuntimeWarning: invalid value encountered in true_divide
  vprop.fa = vprop.fa / numpy.linalg.norm(vprop.fa)


Time Taken: 9.770009994506836s


In [11]:
start=time.time()
# pagerank
g_pagerank = pagerank(g)
pagerank_dic = {}
for i in range(0, len(g_pagerank.a)):
    pagerank_dic[g.vertex_index[i]] = g_pagerank.a[i]
end = time.time()
print("Time Taken: " + str(end - start) +"s")

Time Taken: 2.790212392807007s


In [12]:
# print centrality metrics
def topTen(metric_name, dic, string=""):
    most = {}
    less = {}
    for i in range(0, 10):
        # +
        n = max(dic, key=dic.get)
        most[n] = dic[n]
        del dic[n]
        # -
        n = min(dic, key=dic.get)
        less[n] = dic[n]
        del dic[n]    
    print(metric_name)
    print(string)
    print("Most central:")
    print(sorted(most.items(), key=lambda x: x[1]))
    print("Least central:")
    print(sorted(less.items(), key=lambda x: x[1]))
    print("="*100)
    
topTen("Degree centrality (IN)", in_dic, "Total zeros: "+str(zero_in))
topTen("Degree centrality (OUT)", out_dic, "Total zeros: "+str(zero_out))
topTen("Betweenness", betweenness_dic, "Total zeros: "+str(zero_betweenness))
topTen("Closeness", closeness_dic, "Total zeros: "+str(zero_closeness))
topTen("Katz", katz_dic)
topTen("PageRank", pagerank_dic)

Degree centrality (IN)
Total zeros: 0
Most central:
[(92, 0.02518328745468916), (79, 0.02796326074514186), (148, 0.029107955629445913), (245, 0.029952849472622715), (85, 0.03115205363713172), (197, 0.03390477228748194), (80, 0.03393202692758442), (191, 0.03436810116922406), (144, 0.037257093020086667), (271, 0.03769316726172631)]
Least central:
[(0, 2.7254640102477445e-05), (17, 2.7254640102477445e-05), (2, 2.7254640102477445e-05), (20, 2.7254640102477445e-05), (22, 2.7254640102477445e-05), (23, 2.7254640102477445e-05), (8, 2.7254640102477445e-05), (26, 2.7254640102477445e-05), (14, 2.7254640102477445e-05), (24, 2.7254640102477445e-05)]
Degree centrality (OUT)
Total zeros: 0
Most central:
[(92, 0.02518328745468916), (79, 0.02796326074514186), (148, 0.029107955629445913), (245, 0.029952849472622715), (85, 0.03115205363713172), (197, 0.03390477228748194), (80, 0.03393202692758442), (191, 0.03436810116922406), (144, 0.037257093020086667), (271, 0.03769316726172631)]
Least central:
[(0, 2.